In [2]:
import pandas as pd 
import numpy as np
import json
import os
import json
import warnings
warnings.filterwarnings('ignore')
import gzip
import faiss

In [4]:

def is_gzipped(file_path):
    with open(file_path, 'rb') as f:
        return f.read(2) == b'\x1f\x8b'
    

print(is_gzipped(r"C:\Users\Rishabh\Desktop\Python New\Amazon Recommender System\product_data\Appliances\Appliances.jsonl")) 

True


In [3]:

reviews = []

with gzip.open(r"C:\Users\Rishabh\Desktop\Python New\Amazon Recommender System\product_data\Appliances\Appliances.jsonl", "rt", encoding="utf-8") as fp:
    for line in fp:
        data_line = json.loads(line.strip())
        reviews.append(data_line)

In [5]:
df1 = pd.DataFrame(reviews)

In [6]:
df1.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Work great,work great. use a new one every month,[],B01N0TQ0OH,B01N0TQ0OH,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1519317108692,0,True
1,5.0,excellent product,Little on the thin side,[],B07DD2DMXB,B07DD37QPZ,AHWWLSPCJMALVHDDVSUGICL6RUCA,1664746863446,0,True
2,5.0,Happy customer!,"Quick delivery, fixed the issue!",[],B082W3Z9YK,B082W3Z9YK,AHZIJGKEWRTAEOZ673G5B3SNXEGQ,1607225435363,0,True
3,5.0,Amazing value,I wasn't sure whether these were worth it or n...,[],B078W2BJY8,B078W2BJY8,AFGUPTDFAWOHHL4LZDV27ERDNOYQ,1534104184306,0,True
4,5.0,Dryer parts,Easy to install got the product expected to re...,[],B08C9LPCQV,B08C9LPCQV,AELFJFAXQERUSMTXJQ6SYFFRDWMA,1620176603754,0,True


In [7]:
df = df1[['rating','parent_asin','user_id']]

In [ ]:
data = []

with open(r"C:\Users\Rishabh\Downloads\Amazon_Project_Data\Beauty\meta_All_Beauty.jsonl (1).gz", "rt", encoding="utf-8") as fp:
    for line in fp:
        data_line = json.loads(line.strip())
        data.append(data_line)

In [7]:
df2 = pd.DataFrame(data)

In [8]:
len(df1['parent_asin'].unique())

112565

In [9]:
len(df2['parent_asin'].unique()) , len(df1['parent_asin'].unique())

(112590, 112565)

In [10]:
new_df =pd.merge(df1,df2, on = 'parent_asin')

In [11]:
df1.shape,df2.shape

((701528, 10), (112590, 14))

In [12]:
new_df = new_df.sample(frac = 0.01)


In [13]:
r_df = new_df[['parent_asin','user_id','rating']]

In [14]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
 
# Define the format
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(r_df[["user_id", "parent_asin", "rating"]], reader)

# Train-test split
trainset, testset = train_test_split(data, test_size=0.2)

# Train SVD model
model = SVD()
model.fit(trainset)

# Predict and evaluate
predictions = model.test(testset)
print("RMSE:", accuracy.rmse(predictions))


RMSE: 1.4851
RMSE: 1.4850501883622036


In [15]:
user_mat = model.pu
item_mat = model.qi 

In [16]:
user_mat = user_mat.astype('float32')
item_mat = item_mat.astype('float32')

In [17]:
user_mat.shape[0],item_mat.shape

(5596, (4560, 100))

In [18]:
user_ids = [trainset.to_raw_uid(i) for i in range(user_mat.shape[0])]
item_ids = [trainset.to_raw_iid(j) for j in range(item_mat.shape[0])]

In [19]:
user_df = pd.DataFrame(user_mat,index=user_ids)

In [20]:
item_df = pd.DataFrame(item_mat,index=item_ids)

In [11]:
user_df

NameError: name 'user_df' is not defined

In [22]:
user= user_df.iloc[400] 
user.shape

(100,)

In [23]:
user_array = np.array(user).reshape(1,-1)

In [24]:
d = item_mat.shape[1]

index = faiss.IndexFlatIP(d)  
index.add(item_mat)
k  = item_mat.shape[0]

distances,indices = index.search(user_array, k) 

In [25]:
top_user_ids  = indices[0]

In [26]:
top_dist  = distances[0]

In [27]:
#for i in top_user_ids:
 #   print(user_df.index[i])

In [28]:
top_user_ids[0]

1772

In [29]:
dist =list(top_dist)

In [30]:
sim_ids = [user_df.index[i] for i in top_user_ids]


In [31]:
user_dist_df = pd.DataFrame(dist,sim_ids,columns=['Distance'])

In [32]:
len(sim_ids)

4560

In [33]:
len(r_df['user_id'].unique())

6990

In [34]:
r_df['parent_asin'][r_df['user_id'] == 'AFR2TVY45HMKVZU2IBCR3DB2ESAQ']

Series([], Name: parent_asin, dtype: object)

In [35]:
mat = {}
for i in sim_ids:
    user_r = dict(zip(r_df['parent_asin'][r_df['user_id'] == i],r_df['rating'][r_df['user_id'] == i]))  
    mat[i] = user_r

In [36]:
#filter_df = r_df[r_df['user_id'].isin(sim_ids)]

In [37]:
#mat = (filter_df.groupby('user_id').apply(lambda x: dict(zip( x['parent_asin'] ,x['rating'] )) ).to_dict())     

In [38]:
user_dist_df

,Distance
AE473N3C62NCTUFKQ3S5MEG7VV6A,0.395182
AGE43YVBZBNDKZXAYUH7T4PRVFMA,0.379780
AFPNS3QJZSTRREDG6AWQPZWA76VA,0.373747
AF62SIYNIKEFQXTYADQ5NWDNUYLQ,0.369976
AFIS2P42MZBJ2KB6KO655YSZKELA,0.365041
...,...
AGRH7ORHR745X5TSZDEO4RRU3HFQ,-0.352158
AGNN2ZWUN2WIUDVC3VCVRE6POMRA,-0.357317
AFFJEOWFZ6HHUIAYUJEKI4ANDLZQ,-0.364813
AHSO6LYXYFBTBYNY4V7R5LSED75Q,-0.368823


In [39]:
df_mat =pd.DataFrame(mat)

In [1]:
df_mat.head()  

NameError: name 'df_mat' is not defined

In [43]:
t_df = df_mat.fillna(0)

In [44]:
t_df

,AE473N3C62NCTUFKQ3S5MEG7VV6A,AGE43YVBZBNDKZXAYUH7T4PRVFMA,AFPNS3QJZSTRREDG6AWQPZWA76VA,AF62SIYNIKEFQXTYADQ5NWDNUYLQ,AFIS2P42MZBJ2KB6KO655YSZKELA,AHUGKUVWBYGBSXW3SSMMR2QNTKNQ,AFO456XNS5YAY3LHNVPLRJUJDCWA,AEBAUHHD6BLR276NRQL4IQZK7AIA,AGHABE2FIHA2R6QH6ZRXB2WOEJCA,AGHL4YKLZXMA6EJVQ3HMA7R7TAEQ,...,AHKTCQ5JJQOYHGAHBPKINIRVKUHQ,AHHRZK4U2MTWGIRT2M3U3SPQ6SZA,AER62CDQFLZ5HUQPV5PELHQBWCUQ,AEKJYOIX4FMCCFGQ7SKXDVIPPJDQ,AH2MORTGCST5G33AMEITA47QDK6Q,AGRH7ORHR745X5TSZDEO4RRU3HFQ,AGNN2ZWUN2WIUDVC3VCVRE6POMRA,AFFJEOWFZ6HHUIAYUJEKI4ANDLZQ,AHSO6LYXYFBTBYNY4V7R5LSED75Q,AENEHO4ZDUMMLNR5CIBZKZZXNKRA
B01M1OFZOG,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B07DM82YBT,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00JQI7JBK,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B010B5ZE4U,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B07STXMKKT,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B01M0DFPV1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
B002D4IBFM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
B07YSFYJSH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
B079TJVNLL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0


In [ ]:
product_scores = np.dot(user_dist_df.T,t_df.T)
product_scores 

In [ ]:
## Similar user ids
for i in top_user_ids:
    print(user_df.index[i])

In [ ]:
user_rating = r_df['rating'][r_df['user_id'] == 'AFER7OPRGQOYGVSQCMVCPFLY4PKA']

In [ ]:
sim_products = [r_df['parent_asin'][r_df['user_id'] == i] for i in sim_ids]

In [45]:
distance_df = pd.DataFrame(distances[0],columns=['Distance'])

In [46]:
indice_df = pd.DataFrame(indices[0],columns = ['Index'])

In [47]:
score_ind = pd.concat([distance_df,indice_df],axis = 1)

In [48]:
score_ind

,Distance,Index
0,0.395182,1772
1,0.379780,2827
2,0.373747,2957
3,0.369976,59
4,0.365041,517
...,...,...
4555,-0.352158,1971
4556,-0.357317,2712
4557,-0.364813,4078
4558,-0.368823,1272


In [ ]:
item_df.index[6424]

In [ ]:
print(new_df['images_y'].iloc[2154][0]['large'])
print(new_df['images_y'].iloc[1037][0]['large'])
print(new_df['images_y'].iloc[6424][0]['large'])
print(new_df['images_y'].iloc[7250][0]['large'])
print(new_df['images_y'].iloc[252][0]['large'])


In [ ]:
print(new_df['title_y'].iloc[2154] )
print(new_df['title_y'].iloc[1037])
print(new_df['title_y'].iloc[6424])
print(new_df['title_y'].iloc[7250])
print(new_df['title_y'].iloc[252])

In [ ]:
#!pip install pyspark

In [ ]:
#from pyspark.sql import SparkSession

In [ ]:
#spark = SparkSession.builder.appName("MyFirstSparkApp").getOrCreate()

In [ ]:
#import os
#print("JAVA_HOME:", os.environ.get("JAVA_HOME"))

In [ ]:
#user_df = pd.DataFrame(user_mat)

In [ ]:
#item_df = pd.DataFrame(item_mat)

In [ ]:
#df_spark = spark.createDataFrame(item_df)

In [ ]:
#spark_user_df = spark.createDataFrame(user_mat)